In [32]:
import numpy as np
import pandas as pd

In [33]:
latent_dim = 100
sigma = 1
sigma_p = 1
sigma_q = 1
mu = 0
rating_path = '../data/ml-latest-small/ratings.csv'

In [112]:
rating_pd = pd.read_csv(rating_path)
num_user = np.unique(rating_pd.values[:, 0]).shape[0] # 610 users
num_movie = np.unique(rating_pd.values[:, 1]).shape[0] # 9724 movies
print(num_user)
print(num_movie)
rating_pd.head()

610
9724


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [113]:
r_matrix = np.zeros((num_user, num_movie)) # 610 x 9724
p_matrix = np.random.normal(mu, sigma_p, (num_user, latent_dim)) # 610 x 100
q_matrix = np.random.normal(mu, sigma_q, (num_movie, latent_dim)) # 9724 x 100

In [114]:
movie_dic = {} 
idx = 0
for movie_id in np.unique(rating_pd.values[:, 1]): 
    movie_dic[movie_id] = idx
    idx += 1

for index, row in rating_pd.iterrows():
    r_matrix[int(row['userId'] - 1), movie_dic[row['movieId']]] = row['rating']

In [115]:
# print(r_matrix)

In [116]:
def MSE (r, p, q):
    p_qt_matrix = np.dot(p, q.T) # predicted matrix: 610x100 x 100x9724 = 610x9724
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return np.sum(result)
    
def L2_loss(sig1, sig2, matrix):
    result = np.power(matrix, 2)
    result = np.sum(result)
    constant = sig1 / float(sig2 * 2)
    return constant * result    

In [117]:
error = MSE(r_matrix, p_matrix, q_matrix) + L2_loss(sigma, sigma_q, q_matrix) + L2_loss(sigma, sigma_p, p_matrix)

In [118]:
error

6122749.3430575775

In [119]:
def mat (r, p, q):
    p_qt_matrix = np.dot(p, q.T)
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return result

In [120]:
learning_rate = 0.000001

for i in range(10):
    
    a_matrix = mat(r_matrix, p_matrix, q_matrix) 
    
    error_p_d = 2*np.dot(a_matrix,np.negative(q_matrix))+0+sigma/2*sigma_p*(2*np.sum(p_matrix))
    
    error_q_d = -2*np.dot(a_matrix.T,p_matrix)+sigma/2*sigma_q*(2*np.sum(q_matrix))+0 

    p_matrix = p_matrix - learning_rate * error_p_d 
    
    q_matrix = q_matrix - learning_rate * error_q_d 
    
    error = MSE(r_matrix, p_matrix, q_matrix)
    
    print(error)

5541784.537824479
5481790.092234419
5424430.583242543
5369550.3325297935
5317005.466554979
5266665.226863091
5218410.528812191
5172132.664252034
5127732.184370369
5085117.93469279
